# Artificial Intelligence Nanodegree
## Machine Translation Project

## Introduction
A deep neural network that functions as part of an end-to-end machine translation pipeline. The pipeline accepts English text as input and return the French translation.

- **Preprocess** - Convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations.
- **Prediction** Run the model on English text.

In [1]:
import collections

import numpy as np
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1194811475330217717
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 371683774630322430
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2541355008
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9698619487449847594
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15909339003377872311
physical_device_desc: "device: XLA_GPU device"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file.

In [3]:
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [4]:
# Load English data
english_sentences = load_data('data/small_vocab_en')
# Load French data
french_sentences = load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


The puncuations have been delimited using spaces. All the text have been converted to lowercase. But the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess
We convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we use those.

We turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function.

In [7]:
def tokenize(x):
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
   
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

We make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    
    # Then, pass it to pad_sentences as the maxlen parameter
    x = pad_sequences(x, maxlen=length, padding="post")
    
    return x

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
Experimenting with various neural network architectures.
- Model 1: simple RNN
- Model 2: a RNN with Embedding
- Model 3: a Bidirectional RNN
- Model 4: an Encoder-Decoder RNN

### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` bridges the gab between the logits from the neural network to the French translation.

In [10]:
epochs = 10

def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.

In [11]:
from keras import Sequential

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    # Build the layers
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(512, return_sequences=True,
                  input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size)))

    model.add(Activation('softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=epochs, validation_split=0.2)

# Print predictions
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 29s 261us/step - loss: 1.8423 - accuracy: 0.5638 - val_loss: nan - val_accuracy: 0.6220
Epoch 2/10
110288/110288 [==============================] - 28s 251us/step - loss: 1.2327 - accuracy: 0.6418 - val_loss: nan - val_accuracy: 0.6544
Epoch 3/10
110288/110288 [==============================] - 27s 247us/step - loss: 1.0772 - accuracy: 0.6674 - val_loss: nan - val_accuracy: 0.6701
Epoch 4/10
110288/110288 [==============================] - 27s 247us/step - loss: 0.9893 - accuracy: 0.6795 - val_loss: nan - val_accuracy: 0.6769
Epoch 5/10
110288/110288 [==============================] - 27s 247us/step - loss: 0.9258 - accuracy: 0.6901 - val_loss: nan - val_accuracy: 0.7016
Epoch 6/10
110288/110288 [==============================] - 27s 247us/step - loss: 0.8739 - accuracy: 0.7017 - val_loss: nan - val_accuracy: 0.7124
Epoch 7/10
110288/110288 [==============================] - 2

### Model 2: Embedding
![RNN](images/embedding.png)
An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

We create a RNN model using embedding.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(Embedding(english_vocab_size, output_sequence_length, input_length=input_shape[1]))
    model.add(GRU(512, return_sequences=True,
                  input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size)))

    model.add(Activation('softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=epochs, validation_split=0.2)

# Print predictions
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

/home/rushil/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 28s 258us/step - loss: 2.8046 - accuracy: 0.4658 - val_loss: nan - val_accuracy: 0.5255
Epoch 2/10
110288/110288 [==============================] - 28s 250us/step - loss: 1.5514 - accuracy: 0.6206 - val_loss: nan - val_accuracy: 0.6948
Epoch 3/10
110288/110288 [==============================] - 28s 250us/step - loss: 0.9682 - accuracy: 0.7427 - val_loss: nan - val_accuracy: 0.7927
Epoch 4/10
110288/110288 [==============================] - 28s 250us/step - loss: 0.6256 - accuracy: 0.8291 - val_loss: nan - val_accuracy: 0.8538
Epoch 5/10
110288/110288 [==============================] - 28s 250us/step - loss: 0.4386 - accuracy: 0.8705 - val_loss: nan - val_accuracy: 0.8831
Epoch 6/10
110288/110288 [==============================] - 28s 250us/step - loss: 0.3515 - accuracy: 0.8897 - val_loss: nan - val_accuracy: 0.8969
Epoch 7/10
110288/110288 [==============================] - 2

### Model 3: Bidirectional RNNs
![RNN](images/bidirectional.png)
RNN can't see the future input, only the past.<br>
We try bidirectional recurrent neural networks as they are able to see the future data.

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(512, return_sequences=True),
                  input_shape=input_shape[1:]))
    
    model.add(TimeDistributed(Dense(french_vocab_size)))

    model.add(Activation('softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=epochs, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 51s 462us/step - loss: 1.5771 - accuracy: 0.6033 - val_loss: nan - val_accuracy: 0.6483
Epoch 2/10
110288/110288 [==============================] - 52s 468us/step - loss: 1.0936 - accuracy: 0.6721 - val_loss: nan - val_accuracy: 0.6854
Epoch 3/10
110288/110288 [==============================] - 50s 454us/step - loss: 0.9487 - accuracy: 0.6986 - val_loss: nan - val_accuracy: 0.7039
Epoch 4/10
110288/110288 [==============================] - 50s 453us/step - loss: 0.8642 - accuracy: 0.7126 - val_loss: nan - val_accuracy: 0.7188
Epoch 5/10
110288/110288 [==============================] - 50s 453us/step - loss: 0.8015 - accuracy: 0.7257 - val_loss: nan - val_accuracy: 0.7317
Epoch 6/10
110288/110288 [==============================] - 50s 453us/step - loss: 0.7448 - accuracy: 0.7398 - val_loss: nan - val_accuracy: 0.7451
Epoch 7/10
110288/110288 [==============================] - 5

### Model 4: Encoder-Decoder
This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [14]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3

        
    model = Sequential()
    # encoder
    model.add(Bidirectional(GRU(512, return_sequences=False),
                  input_shape=input_shape[1:]))
    model.add(Dense(french_vocab_size))
    model.add(Activation('relu'))
    
    # decoder
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(512, return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size)))
    model.add(Activation('softmax'))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Build the model
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Train the neural network
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=epochs, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 74s 672us/step - loss: 2.0114 - accuracy: 0.5340 - val_loss: nan - val_accuracy: 0.6068
Epoch 2/10
110288/110288 [==============================] - 73s 658us/step - loss: 1.3598 - accuracy: 0.6230 - val_loss: nan - val_accuracy: 0.6377
Epoch 3/10
110288/110288 [==============================] - 73s 658us/step - loss: 1.2256 - accuracy: 0.6518 - val_loss: nan - val_accuracy: 0.6646
Epoch 4/10
110288/110288 [==============================] - 73s 657us/step - loss: 1.1486 - accuracy: 0.6682 - val_loss: nan - val_accuracy: 0.6674
Epoch 5/10
110288/110288 [==============================] - 73s 658us/step - loss: 1.0470 - accuracy: 0.6884 - val_loss: nan - val_accuracy: 0.6931
Epoch 6/10
110288/110288 [==============================] - 73s 657us/step - loss: 1.0097 - accuracy: 0.6890 - val_loss: nan - val_accuracy: 0.7070
Epoch 7/10
110288/110288 [==============================] - 7

### Model 5: Custom

In [15]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    
    model = Sequential()
    # encoder
    model.add(Embedding(input_dim=english_vocab_size, output_dim=512,
                        input_length=input_shape[1:][0]))
    model.add(Bidirectional(GRU(512),
                  input_shape=input_shape[1:]))
    model.add(Dense(french_vocab_size))
    model.add(Activation('relu'))
    
    # decoder
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(512, return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size)))
    model.add(Activation('softmax'))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

print('Final Model Loaded')

Final Model Loaded


## Prediction

In [16]:
def final_predictions(x, y, x_tk, y_tk):
    # Train neural network using model_final
    model = model_final(x.shape,
                        y.shape[1],
                        len(x_tk.word_index),
                        len(y_tk.word_index))
    
    model.fit(x, y, batch_size=512, epochs=epochs, validation_split=0.2)


    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

/home/rushil/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 103s 931us/step - loss: 1.9114 - accuracy: 0.5591 - val_loss: nan - val_accuracy: 0.6714
Epoch 2/10
110288/110288 [==============================] - 100s 911us/step - loss: 1.0044 - accuracy: 0.7128 - val_loss: nan - val_accuracy: 0.7441
Epoch 3/10
110288/110288 [==============================] - 100s 911us/step - loss: 0.7475 - accuracy: 0.7756 - val_loss: nan - val_accuracy: 0.8087
Epoch 4/10
110288/110288 [==============================] - 100s 911us/step - loss: 0.5359 - accuracy: 0.8380 - val_loss: nan - val_accuracy: 0.8634
Epoch 5/10
110288/110288 [==============================] - 100s 911us/step - loss: 0.3548 - accuracy: 0.8951 - val_loss: nan - val_accuracy: 0.9125
Epoch 6/10
110288/110288 [==============================] - 100s 911us/step - loss: 0.2280 - accuracy: 0.9347 - val_loss: nan - val_accuracy: 0.9478
Epoch 7/10
110288/110288 [=============================